# 完成 flask_app.py 的 TODO

In [ ]:
!cat flask_app.py

# 透過 gunicorn 執行 flask_app.py  
```
gunicorn -b localhost:8080 flask_app:app
```

In [ ]:
# 測試 flask_app.py 沒問題
!curl http://localhost:8080

# Gunicorn work type
參考 [淺談 Gunicorn 各個 worker type 適合的情境](https://medium.com/@genchilu/%E6%B7%BA%E8%AB%87-gunicorn-%E5%90%84%E5%80%8B-worker-type-%E9%81%A9%E5%90%88%E7%9A%84%E6%83%85%E5%A2%83-490b20707f28)
## sync
底層實作是每個請求都由一個 process 處理。(gunicorn 預設為此 work type)
## gthread
則是每個請求都由一個 thread 處理。
## eventlet、gevent、tarnado
底層則是利用非同步 IO 讓一個 process 在等待 IO 回應時繼續處理下個請求。

## 用 process 處理請求
當 gunicorn worker type 使用 sync 時，web 啟動時會預先開好對應數量的 process 處理請求，理論上 concurrency 的上限等同於 worker 數量，worker 為 process 的數量，參數為 -w  
```
gunicorn -w 1 -k sync -b localhost:8080 flask_app:app
```
這種類型的好處是錯誤隔離高，一個 process 掛掉只會影響該 process 當下服務的請求，而不會影響其他請求。  
壞處則為 process 資源開銷較大，開太多 worker 時對記憶體或 CPU 的影響很大，因此 concurrency 理論上限極低。  

In [ ]:
# 用 siege 分別對 IO bound task 和 CPU bound task 發出 2 個請求可以明顯看到第二個 request 被第一個 request 阻塞
! siege -c 2 -r 1 http://localhost:8080/ioTask -v
! siege -c 2 -r 1 http://localhost:8080/cpuTask -v

## 用 thread 處理請求
當 gunicorn worker type 用 gthread 時，可額外加參數 --thread 指定每個 process 能開的 thread 數量，此時 concurrency 的上限為 worker 數量乘以給個 worker 能開的 thread 數量  
```
gunicorn -w 1 -k gthread --thread=2 -b localhost:8080 flask_app:app
```
這種類型的 worker 好處是 concurrency 理論上限會比 process 高，壞處依然是 thread 數量，OS 中 thread 數量是有限的，過多的 thread 依然會造成系統負擔。

In [ ]:
# 用 siege 分別對 IO bound task 和 CPU bound task 發出 4 個請求可以明顯看到第三個請求以後才會被阻塞
! siege -c 4 -r 1 http://localhost:8080/ioTask -v
! siege -c 4 -r 1 http://localhost:8080/cpuTask -v

## 用非同步 IO 處理每個請求
當 gunicorn worker type 用 eventlet、gevent、tarnado 等類型時，每個請求都由同一個 process 處理，而當遇到 IO 時該 process 不會等 IO 回應，會繼續處理下個請求直到該 IO 完成，理論上 concurrency 無上限。
```
gunicorn -w 1 -k gevent -b localhost:8080 flask_app:app
```

In [ ]:
# 用 siege 對 IO bound task 發出 10 個請求可以明顯看到沒有任何請求被阻塞
! siege -c 10 -r 1 http://localhost:8080/ioTask -v

In [ ]:
! siege -c 10 -r 1 http://localhost:8080/cpuTask -v

使用非同步類型的 worker 好處和壞處非常明顯，對 IO bound task 的高效能，但在 CPU bound task 會不如 thread。